# RAG (검색 증강 생성) : FAISS 사용
: https://www.promptingguide.ai/kr/techniques/rag

문서를 벡터로 변환(임베딩) : 텍스트 데이터를 고차원 벡터 공간으로 매핑하는 작업

### FAISS (Facebook AI Similarity Search)
Facebook AI 팀에서 개발한 대규모 벡터 검색 라이브러리로, 빠르고 효율적인 최근접 이웃 검색(Nearest Neighbor Search, NNS) 을 수행하는 데 사용된다. 
<br>
특히, 고차원 벡터 검색을 최적화하여 딥러닝 임베딩 검색, 추천 시스템, RAG(Retrieval-Augmented Generation) 등에서 널리 활용된다

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [4]:
# ! pip install faiss-cpu
import faiss
import numpy as np

# 샘플 문서데이터 (텍스트)
documents = [
    "OpenAI는 인공지능 연구소입니다.",
    "FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.",
    "GPT는 자연어 처리를 위한 강력한 AI 모델입니다.",
    "RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다."    
]

In [6]:
# 문서를 벡터로 변환 (임베딩)
# https://platform.openai.com/docs/pricing
def get_embedding(text):
    response = client.embeddings.create(
        model = 'text-embedding-3-small',   # text-embedding-ada-002, text-embedding-3-large
        input = text        
    )
    return np.array(response.data[0].embedding)

# 모든 문서의 임베딩 생성
document_embeddings = np.array([get_embedding(doc) for doc in documents ])
print(document_embeddings)

[[-0.00736527 -0.00445131  0.0193053  ...  0.00990422 -0.00136497
   0.00519902]
 [-0.01358794  0.00780865 -0.02596267 ...  0.02078999 -0.03368309
  -0.01462468]
 [-0.01112392  0.07651846  0.03827878 ... -0.02164178 -0.00357275
   0.00447449]
 [ 0.01525772  0.02207769  0.00052901 ... -0.01211735 -0.01122917
  -0.03973562]]


In [8]:
print(document_embeddings[0])        # "OpenAI는 인공지능 연구소입니다."
print(len(document_embeddings[0]))   # 1536

[-0.00736527 -0.00445131  0.0193053  ...  0.00990422 -0.00136497
  0.00519902]
1536


In [12]:
# FAISS 벡터 인덱스 생성
dimension = document_embeddings.shape[1]   
print(dimension)    # 1536
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

1536


In [14]:
#  사용자 질문 입력 및 벡터 변환
query = "RAG의 핵심 구성 요소는 무엇인가?"
query_embedding = get_embedding(query)
print(query_embedding)

[ 0.01566273  0.00232661 -0.03758604 ... -0.0011633  -0.01976138
 -0.01448042]


In [19]:
#  FAISS를 이용한 검색( 가장 유사한 문서 찾기)
k = 2  # 가장 유사한 2개의 문서 검색
distances,indices = index.search(query_embedding.reshape(1,-1),k)
print(distances, indices)

[[1.0512233 1.575014 ]] [[3 1]]


In [27]:
# 배열의 reshape
a = np.arange(12)
print(a.shape)  # 1차원 배열
print(a)
a2 = a.reshape(3,4)
print(a2.shape)  # 2차원 배열
print(a2)
a3 = a.reshape(2,3,2) 
print(a3.shape)  # 3차원 배열
print(a3)

b = a.reshape(3,-1)  # -1은 shape을 자동으로 계산
print(b,b.shape)

(12,)
[ 0  1  2  3  4  5  6  7  8  9 10 11]
(3, 4)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
(2, 3, 2)
[[[ 0  1]
  [ 2  3]
  [ 4  5]]

 [[ 6  7]
  [ 8  9]
  [10 11]]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]] (3, 4)
